In [12]:
import sqlite3
import pandas as pd
import settings
import datetime as dt

In [2]:
# trades check
exchanges = settings.exchanges
conn = sqlite3.connect('test.db')

for k, v in exchanges.items():
    exchange_name = k
    exchange = v
    table_name = exchange_name + "_trades"
    # print(df)
    print(pd.read_sql_query("SELECT * from {}".format(table_name),conn))



     index            utc                 time  \
0        0  1587566826553  2020-04-22 15:47:06   
1        1  1587566826553  2020-04-22 15:47:06   
2        2  1587566826553  2020-04-22 15:47:06   
3        3  1587567287153  2020-04-22 15:54:47   
4        4  1587621782248  2020-04-23 07:03:02   
..     ...            ...                  ...   
156      5  1588100057178  2020-04-28 19:54:17   
157      6  1588100060962  2020-04-28 19:54:20   
158      7  1588100071889  2020-04-28 19:54:31   
159      8  1588100274338  2020-04-28 19:57:54   
160      9  1588100274612  2020-04-28 19:57:54   

                                 trade_id  \
0    230b3317-84a8-11ea-b93c-0242ac11000b   
1    230b3316-84a8-11ea-b93c-0242ac11000b   
2    230b3315-84a8-11ea-b93c-0242ac11000b   
3    35952f01-84a9-11ea-b93c-0242ac11000b   
4    1730c764-8528-11ea-b93c-0242ac11000b   
..                                    ...   
156  a9437c06-8981-11ea-b93c-0242ac11000b   
157  ab84e0a9-8981-11ea-b93c-0242ac1100

In [5]:
# bal & orderbook check 
for k, v in exchanges.items():
    exchange_name = k
    exchange = v
    table_name = exchange_name + "_bal_orderbook"
    df = pd.read_sql_query("SELECT * from {}".format(table_name),conn)
    print(pd.read_sql_query("SELECT * from {}".format(table_name),conn).tail())


      index               time           utc exchange     pair        fx  \
2120      0  20-04-28 19:26:01  1.588098e+09   bitbay  ETH/PLN  4.199435   
2121      0  20-04-28 19:27:02  1.588098e+09   bitbay  ETH/PLN  4.199320   
2122      0  20-04-28 19:28:02  1.588098e+09   bitbay  ETH/PLN  4.199210   
2123      0  20-04-28 19:29:01  1.588099e+09   bitbay  ETH/PLN  4.199230   
2124      0  20-04-28 19:31:02  1.588099e+09   bitbay  ETH/PLN  4.199220   

      bal_fiat_free  bal_fiat_used  bal_fiat_total  bal_eth_free  ...  \
2120       61798.80       56362.19       118160.99    122.296704  ...   
2121       61798.80       56362.19       118160.99    122.296704  ...   
2122       61798.87       56362.12       118160.99    122.296704  ...   
2123       61798.87       56362.12       118160.99    122.296704  ...   
2124       61798.80       56362.19       118160.99    122.296704  ...   

      1_bid_sz  2_bid_sz  3_bid_sz  4_bid_sz  5_bid_sz   1_ask_sz  2_ask_sz  \
2120  0.012005   0.72993 

In [133]:
# merge check 
for k, v in exchanges.items():
    exchange_name = k
    exchange = v
    table_name = exchange_name + "_merge_td_bal_ods"
    df = pd.read_sql_query("SELECT * from {}".format(table_name),conn)
#     print(df)


In [134]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def cal_bal_usd(x):
    return (x['bal_fiat_total'] + x['bal_eth_total'] * x['1_bid_px'])/x['fx_x']

def pnl_to_string(amt, pct, rlt_amt, rlt_pct):
    amt =  '$' + amt.round().to_string().split()[1] + '/'
    pct =  (100 * pct).round(4).to_string().split()[1] + '%/'
    rlt_amt =  '$' + rlt_amt.round().to_string().split()[1] + '/'
    rlt_pct =  (100 * rlt_pct).round(4).to_string().split()[1] + '%'
    s= amt + pct + rlt_amt + rlt_pct
    return s

def pnl(df, time):
    df['time_x'] = df['time_x'].apply(lambda x: dt.datetime.strptime(x,"%y-%m-%d %H:%M:%S"))
    now = df['time_x'].iloc[-1]
    previous = now - dt.timedelta(hours = time)
    index = nearest(df['time_x'], previous)
    condition = df['time_x'] == index
    x_data = df[condition] # previous time data
    y_data = df.iloc[-1] # current time data
    x_bal = cal_bal_usd(x_data)
    y_bal = cal_bal_usd(y_data)
    amt = y_bal - x_bal
    pct = amt/x_bal
    mkt_amt = (y_data['1_bid_px'] - x_data['1_bid_px'])/x_data['fx_x']
    print(mkt_amt)
    mkt_pct = mkt_amt/x_data['1_bid_px']
    rlt_amt = amt - mkt_amt * 0.5
    rlt_pct = pct - mkt_pct * 0.5
    s = pnl_to_string(amt, pct, rlt_amt, rlt_pct)
    return y_data['1_bid_px']
    

In [135]:
s = pnl(df, 24000)
s

0    2.795239
dtype: float64


3007000.0

In [140]:
df['1_bid_px'][0] - df['1_bid_px'][1000]

-3000.0

In [10]:
# OpenOrder check 
for k, v in exchanges.items():
    exchange_name = k
    exchange = v
    table_name = exchange_name + "_open_order"
    # print(df)
    print(pd.read_sql_query("SELECT * from {}".format(table_name),conn))


      index               time           utc exchange     pair        fx  \
0         0  20-04-26 09:21:02  1.587889e+09   bitbay  ETH/PLN  4.190604   
1         1  20-04-26 09:21:02  1.587889e+09   bitbay  ETH/PLN  4.190604   
2         0  20-04-26 09:22:02  1.587889e+09   bitbay  ETH/PLN  4.190604   
3         0  20-04-26 09:23:02  1.587889e+09   bitbay  ETH/PLN  4.190604   
4         1  20-04-26 09:23:02  1.587889e+09   bitbay  ETH/PLN  4.190604   
...     ...                ...           ...      ...      ...       ...   
1106      0  20-04-27 06:18:01  1.587965e+09   bitbay  ETH/PLN  4.166320   
1107      1  20-04-27 06:18:01  1.587965e+09   bitbay  ETH/PLN  4.166320   
1108      2  20-04-27 06:18:01  1.587965e+09   bitbay  ETH/PLN  4.166320   
1109      0  20-04-27 06:19:01  1.587965e+09   bitbay  ETH/PLN  4.166055   
1110      1  20-04-27 06:19:01  1.587965e+09   bitbay  ETH/PLN  4.166055   

                              OpenOrder_id  OpenOrder_timestamp  \
0     3e5243f7-879f-